# Star Schema Design and Implementation in Jupyter Notebook


In [65]:
import pandas as pd
import json
import uuid
import re
from datetime import datetime
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side

def extract_json_data(row):
    """Extract JSON data from raw_content with error handling"""
    try:
        content = row.get('raw_content', '{}')
        if pd.isna(content) or not content:
            return {}
        return json.loads(content)
    except json.JSONDecodeError:
        try:
            # Try to fix malformed JSON
            fixed = content.replace("'", '"').replace('None', 'null')
            return json.loads(fixed)
        except:
            return {}
    except Exception:
        return {}


def load_and_prepare_data(file_path):
    """Load and prepare the raw data"""
    raw_df = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Ensure required columns exist
    required_cols = ['id', 'comm_type', 'raw_content', 'subject']
    for col in required_cols:
        if col not in raw_df.columns:
            raw_df[col] = None
    
    # Extract JSON data
    json_data = raw_df.apply(extract_json_data, axis=1)
    json_df = pd.json_normalize(json_data)
    
    # Combine with raw data, avoiding duplicate columns
    for col in json_df.columns:
        if col not in raw_df.columns:
            raw_df[col] = json_df[col]
    
    return raw_df

def create_dimension_tables(raw_df):
    """Create all dimension tables with enhanced name extraction"""
    # dim_comm_type
    comm_types = raw_df['comm_type'].dropna().unique()
    dim_comm_type = pd.DataFrame({
        'comm_type': comm_types,
        'comm_type_id': range(1, len(comm_types) + 1)
    })
    
    # dim_subject
    subjects = raw_df['subject'].dropna().unique()
    dim_subject = pd.DataFrame({
        'subject': subjects,
        'subject_id': range(1, len(subjects) + 1)
    })
    
    # dim_user - collect all unique users from various fields
    user_records = []
    seen_emails = set()
    
    speaker_data = {}
    for _, row in raw_df.iterrows():
        content = extract_json_data(row)
        if not content:
            continue
        
        for speaker in content.get('speakers', []):
            if isinstance(speaker, dict):
                name = speaker.get('name')
                email = speaker.get('email', '').lower().strip()
                if email:  # Only store if we have an email
                    speaker_data[email] = {
                        'name': name,
                        'location': speaker.get('location'),
                        'displayName': speaker.get('displayName'),
                        'phoneNumber': speaker.get('phoneNumber')
                    }
    
    # Second pass: Process all data sources
    for _, row in raw_df.iterrows():
        content = extract_json_data(row)
        if not content:
            continue
        
        # Process all potential email sources
        email_sources = []
        
        # Speakers
        email_sources.extend(
            (speaker.get('email', '').lower().strip(), 'speaker') 
            for speaker in content.get('speakers', []) 
            if isinstance(speaker, dict)
        )
        
        # Participants
        for participant in content.get('participants', []):
            if isinstance(participant, dict):
                email = participant.get('email', '').lower().strip()
                if email:
                    email_sources.append((email, 'participant'))
            elif isinstance(participant, str):
                email = participant.lower().strip()
                if email:
                    email_sources.append((email, 'participant'))
        
        # Meeting attendees
        email_sources.extend(
            (attendee.get('email', '').lower().strip(), 'attendee') 
            for attendee in content.get('meeting_attendees', []) 
            if isinstance(attendee, dict)
        )
        
        # Organizer
        organizer_email = content.get('organizer_email', '').lower().strip()
        if organizer_email:
            email_sources.append((organizer_email, 'organizer'))
        
        # Create user records for unique emails
        for email, source in email_sources:
            if email and email not in seen_emails:
                seen_emails.add(email)
                
                
                user_records.append({
                    'name': name,
                    'email': email,
                    'location': speaker_data.get(email, {}).get('location'),
                    'displayName': speaker_data.get(email, {}).get('displayName'),
                    'phoneNumber': speaker_data.get(email, {}).get('phoneNumber')
                })
    
    # Create final dim_user table
    dim_user_columns = ['user_id', 'name', 'email', 'location', 'displayName', 'phoneNumber']
    if user_records:
        dim_user = pd.DataFrame(user_records)
        dim_user['user_id'] = [str(uuid.uuid4()) for _ in range(len(dim_user))]
    else:
        dim_user = pd.DataFrame(columns=dim_user_columns)
    
    # Ensure all columns exist
    for col in dim_user_columns[1:]:
        if col not in dim_user.columns:
            dim_user[col] = None
    
    # Other dimensions
    dim_calendar = pd.DataFrame({
        'raw_calendar_id': raw_df['calendar_id'].dropna().unique(),
        'calendar_id': range(1, len(raw_df['calendar_id'].dropna().unique()) + 1)
    }) if 'calendar_id' in raw_df.columns else pd.DataFrame(columns=['raw_calendar_id', 'calendar_id'])
    
    dim_audio = pd.DataFrame({
        'raw_audio_url': raw_df['audio_url'].dropna().unique(),
        'audio_id': range(1, len(raw_df['audio_url'].dropna().unique()) + 1)
    }) if 'audio_url' in raw_df.columns else pd.DataFrame(columns=['raw_audio_url', 'audio_id'])
    
    dim_video = pd.DataFrame(columns=['raw_video_url', 'video_id'])
    
    dim_transcript = pd.DataFrame({
        'raw_transcript_url': raw_df['transcript_url'].dropna().unique(),
        'transcript_id': range(1, len(raw_df['transcript_url'].dropna().unique()) + 1)
    }) if 'transcript_url' in raw_df.columns else pd.DataFrame(columns=['raw_transcript_url', 'transcript_id'])
    
    return {
        'dim_comm_type': dim_comm_type,
        'dim_subject': dim_subject,
        'dim_user': dim_user,
        'dim_calendar': dim_calendar,
        'dim_audio': dim_audio,
        'dim_video': dim_video,
        'dim_transcript': dim_transcript
    }

def create_fact_table(raw_df, dimensions):
    """Create the fact communication table"""
    # Initialize with required columns
    fact_columns = [
        'comm_id', 'raw_id', 'source_id', 'comm_type_id', 'subject_id',
        'calendar_id', 'audio_id', 'video_id', 'transcript_id', 'datetime_id',
        'ingested_at', 'processed_at', 'is_processed', 'raw_title', 'raw_duration'
    ]
    
    if raw_df.empty:
        return pd.DataFrame(columns=fact_columns)
    
    # Create base dataframe with only the columns we need
    base_cols = ['id', 'comm_type', 'subject', 'source_id', 'ingested_at', 'processed_at', 'is_processed']
    fact_df = raw_df[[col for col in base_cols if col in raw_df.columns]].copy()
    
    # Rename columns
    fact_df = fact_df.rename(columns={
        'id': 'comm_id',
        'comm_type': 'comm_type',
        'subject': 'subject',
        'source_id': 'source_id'
    })
    
    # Add fields from JSON data
    json_fields = {
        'raw_id': 'id',
        'raw_title': 'title',
        'raw_duration': 'duration',
        'datetime_id': 'dateString',
        'raw_calendar_id': 'calendar_id',
        'audio_url': 'audio_url',
        'transcript_url': 'transcript_url'
    }
    
    for new_col, old_col in json_fields.items():
        if old_col in raw_df.columns:
            if new_col == 'datetime_id':
                fact_df[new_col] = pd.to_datetime(raw_df[old_col], errors='coerce')
            else:
                fact_df[new_col] = raw_df[old_col].copy()
        else:
            fact_df[new_col] = None
    
    # Merge with dimension tables carefully
    for dim_name, dim_col, fact_col in [
        ('dim_comm_type', 'comm_type', 'comm_type'),
        ('dim_subject', 'subject', 'subject'),
        ('dim_calendar', 'raw_calendar_id', 'raw_calendar_id'),
        ('dim_audio', 'raw_audio_url', 'audio_url'),
        ('dim_transcript', 'raw_transcript_url', 'transcript_url')
    ]:
        if dim_name in dimensions and not dimensions[dim_name].empty and fact_col in fact_df.columns:
            # Drop duplicate columns before merge
            dim_df = dimensions[dim_name].drop(columns=[c for c in dimensions[dim_name].columns if c in fact_df.columns and c != dim_col])
            fact_df = fact_df.merge(
                dim_df,
                left_on=fact_col,
                right_on=dim_col,
                how='left'
            )
    
    # Add video_id (empty)
    fact_df['video_id'] = None
    
    # Ensure we have all required columns
    for col in fact_columns:
        if col not in fact_df.columns:
            fact_df[col] = None
    
    # Select final columns
    return fact_df[fact_columns]

def create_bridge_table(raw_df, dim_user):
    """Create bridge table between communications and users"""
    bridge_columns = ['comm_id', 'user_id', 'isSpeaker', 'isParticipant', 'isAttendee', 'isOrganiser']
    
    if raw_df.empty or dim_user.empty:
        return pd.DataFrame(columns=bridge_columns)
    
    bridge_records = []
    
    for _, row in raw_df.iterrows():
        comm_id = row['id']
        content = extract_json_data(row)
        if not content:
            continue
        
        # Process speakers
        for speaker in content.get('speakers', []):
            email = speaker.get('email', '').lower().strip() if isinstance(speaker, dict) else ''
            if email:
                bridge_records.append({
                    'comm_id': comm_id,
                    'email': email,
                    'isSpeaker': True,
                    'isParticipant': False,
                    'isAttendee': False,
                    'isOrganiser': False
                })
        
        # Process participants
        for participant in content.get('participants', []):
            if isinstance(participant, dict):
                email = participant.get('email', '').lower().strip()
            elif isinstance(participant, str):
                email = participant.lower().strip()
            else:
                email = ''
            
            if email:
                bridge_records.append({
                    'comm_id': comm_id,
                    'email': email,
                    'isSpeaker': False,
                    'isParticipant': True,
                    'isAttendee': False,
                    'isOrganiser': False
                })
        
        # Process meeting attendees
        for attendee in content.get('meeting_attendees', []):
            if isinstance(attendee, dict):
                email = attendee.get('email', '').lower().strip()
                if email:
                    bridge_records.append({
                        'comm_id': comm_id,
                        'email': email,
                        'isSpeaker': False,
                        'isParticipant': False,
                        'isAttendee': True,
                        'isOrganiser': False
                    })
        
        # Process organizer
        organizer_email = content.get('organizer_email', '').lower().strip()
        if organizer_email:
            bridge_records.append({
                'comm_id': comm_id,
                'email': organizer_email,
                'isSpeaker': False,
                'isParticipant': False,
                'isAttendee': True,
                'isOrganiser': True
            })
    
    # Create bridge table
    if not bridge_records:
        return pd.DataFrame(columns=bridge_columns)
    
    bridge_df = pd.DataFrame(bridge_records)
    
    # Merge with dim_user to get user_id
    if not dim_user.empty and 'email' in dim_user.columns:
        bridge_df = bridge_df.merge(
            dim_user[['user_id', 'email']].drop_duplicates(),
            on='email',
            how='left'
        )
    else:
        bridge_df['user_id'] = None
    
    # Select final columns
    return bridge_df[bridge_columns].drop_duplicates()

def save_to_excel(data, output_file):
    """Save all tables to Excel with formatting"""
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for sheet_name, df in data.items():
            # Skip empty DataFrames
            if df.empty:
                continue
                
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Apply formatting
            worksheet = writer.sheets[sheet_name]
            
            # Set column widths
            for column in worksheet.columns:
                max_length = max(
                    len(str(cell.value)) if cell.value else 0
                    for cell in column
                )
                adjusted_width = (max_length + 2) * 1.2
                worksheet.column_dimensions[column[0].column_letter].width = adjusted_width
            
            # Format header
            for cell in worksheet[1]:
                cell.font = Font(bold=True)
                cell.alignment = Alignment(horizontal='center')
                cell.border = Border(bottom=Side(border_style='thin', color='000000'))

def main():
    input_file = "raw_data.xlsx"
    output_file = "final_output_with_names.xlsx"
    
    print("Loading data...")
    raw_df = load_and_prepare_data(input_file)
    
    print("Creating dimension tables...")
    dimensions = create_dimension_tables(raw_df)
    
    print("Creating fact table...")
    fact_table = create_fact_table(raw_df, dimensions)
    
    print("Creating bridge table...")
    bridge_table = create_bridge_table(raw_df, dimensions['dim_user'])
    
    # Combine all tables
    all_tables = {
        'dim_comm_type': dimensions['dim_comm_type'],
        'dim_subject': dimensions['dim_subject'],
        'dim_user': dimensions['dim_user'],
        'dim_calendar': dimensions['dim_calendar'],
        'dim_audio': dimensions['dim_audio'],
        'dim_video': dimensions['dim_video'],
        'dim_transcript': dimensions['dim_transcript'],
        'fact_communication': fact_table,
        'bridge_comm_user': bridge_table
    }
    
    print("Saving to Excel...")
    save_to_excel(all_tables, output_file)
    
    print("\nStar schema successfully created with:")
    print(f"- {len(dimensions['dim_user'])} users in dim_user")
    print(f"- {len(fact_table)} communications in fact_communication")
    print(f"- {len(bridge_table)} relationships in bridge_comm_user")
    print(f"Output saved to {output_file}")

if __name__ == "__main__":
    main()

Loading data...
Creating dimension tables...
Creating fact table...
Creating bridge table...
Saving to Excel...

Star schema successfully created with:
- 253 users in dim_user
- 15 communications in fact_communication
- 296 relationships in bridge_comm_user
Output saved to final_output_with_names.xlsx
